In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## **Motor Vehicle Thefts**

In [ ]:
from PIL import Image
img =Image.open("img/cars_lot.jpg")
img